Before you turn this problem in, make sure everything runs as expected. First, **restart the kernel** (in the menubar, select Kernel$\rightarrow$Restart) and then **run all cells** (in the menubar, select Cell$\rightarrow$Run All).

Make sure you fill in any place that says `YOUR CODE HERE` or "YOUR ANSWER HERE", as well as your name and collaborators below:

In [ ]:
NAME = "Lucas Resende Pellegrinelli Machado"
COLLABORATORS = ""

---

# Exercício Prático 6: Regressão Linear Simples

Neste exercício vamos analisar o conjunto de dados ```Cereals```, que contém uma lista de marcas de cereais com dados nutricionais e avaliações feitas por consumidores. Vamos utilizar a regressão linear simples para descobrir qual dos fatores nutricionais melhor explica as avaliações. 

## Carregando os dados

Iremos carregar os dados usando a biblioteca ```pandas```. Não se preocupe se você não conhece a biblioteca, pois o nosso objetivo é apenas extrair a matriz de dados $X$. Segue uma descrição do dataset, retirada [daqui](http://statweb.stanford.edu/~owen/courses/202/Cereals.txt).

* Datafile Name: Cereals
* Datafile Subjects: Food , Health
* Story Names: Healthy Breakfast
* Reference: Data available at many grocery stores
* Authorization: free use
* Description: Data on several variable of different brands of cereal.

A value of -1 for nutrients indicates a missing observation.
Number of cases: 77
Variable Names:

  1. Name: Name of cereal
  2. mfr: Manufacturer of cereal where A = American Home Food Products; G =
     General Mills; K = Kelloggs; N = Nabisco; P = Post; Q = Quaker Oats; R
     = Ralston Purina
  3. type: cold or hot
  4. calories: calories per serving
  5. protein: grams of protein
  6. fat: grams of fat
  7. sodium: milligrams of sodium
  8. fiber: grams of dietary fiber
  9. carbo: grams of complex carbohydrates
  10. sugars: grams of sugars
  11. potass: milligrams of potassium
  12. vitamins: vitamins and minerals - 0, 25, or 100, indicating the typical percentage of FDA recommended
  13. shelf: display shelf (1, 2, or 3, counting from the floor)
  14. weight: weight in ounces of one serving
  15. cups: number of cups in one serving
  16. rating: a rating of the cereals

In [ ]:
import pandas as pd
df = pd.read_table('cereal.txt',sep='\s+',index_col='name')
df

A seguir iremos remover as linhas correspondentes aos cereais que possuem dados faltantes, representados pelo valor -1.
Também iremos remover as colunas com dados categóricos 'mfr' e 'type', e os dados numéricos, 'shelf', 'weight' e 'cups'.

In [ ]:
import numpy as np
new_df = df.replace(-1,np.nan)
new_df = new_df.dropna()
new_df = new_df.drop(['mfr','type','shelf','weight','cups'],axis=1)
new_df

Finalmente, iremos converter os dados nutricionais numéricos de ```new_df``` para uma matriz ```dados``` e as avaliações (ratings) para um vetor $y$. Os nomes dos cereais serão salvos em uma lista ```cereral_names``` e os nomes das colunas em uma lista ```col_names```.

In [ ]:
cereral_names = list(new_df.index)
print('Cereais:',cereral_names)
col_names = list(new_df.columns)
print('Colunas:',col_names)

dados = new_df.drop('rating', axis=1).values
print('As dimensões de dados são:',dados.shape)
y = new_df['rating'].values
print('As dimensões de y são:',y.shape)

## Estimando os parâmetros da regressão linear simples

Qual será a relação entre a avaliação $y$ e o número de calorias $x$ de um cereal? Para responder esta pergunta, considere uma regressão linear simples
$$
y = \beta_0 + \beta_1 x.
$$
Para encontrar os coeficientes $\beta_0$ e $\beta_1$ utilizando o método dos mínimos quadrados, basta resolver o sistema
$$
\begin{bmatrix}
n & \sum_i x^{(i)} \\
\sum_i x^{(i)} & \sum_i (x^{(i)})^2
\end{bmatrix}
\begin{bmatrix}
\beta_0 \\ \beta_1
\end{bmatrix}
=
\begin{bmatrix}
\sum_i y^{(i)} \\ \sum_i x^{(i)} y^{(i)}
\end{bmatrix}
$$

Portanto, para encontrar $\beta_0$ e $\beta_1$, você precisa
1. Calcular a matriz
$$
A = \begin{bmatrix}
n & \sum_i x^{(i)} \\
\sum_i x^{(i)} & \sum_i (x^{(i)})^2
\end{bmatrix}
$$
e o vetor
$$
c = \begin{bmatrix}
\sum_i y^{(i)} \\ \sum_i x^{(i)} y^{(i)}
\end{bmatrix}
$$
2. Resolver $A \beta = c$, onde $\beta$ é o vetor de coeficientes.

**Exercício:** Encontre os coeficientes $\beta_0$ e $\beta_1$ quando a variável independente é ```calories```. Dica: Esta variável está armazenada na primeira coluna da matriz ```dados```.

In [ ]:
### Passo 0: obtenha o vetor de observações x (~1 linha)
x = dados[:,0]

def regressaoLinearSimples(x, y):
    ### Passo 1: obtenha o número de observações n (~1 linha)
    n = x.shape[0]
    
    ### Passo 2: calcule a matriz A e o vetor c(~2 linhas)
    A = [[n, np.sum(x)], [np.sum(x), np.sum(x**2)]]
    c = [np.sum(y), np.sum(x * y)]

    ### Passo 3: resolva A beta = c (~1 linha, usando np.linalg.solve)
    beta = np.linalg.solve(A, c)

    return beta

beta = regressaoLinearSimples(x,y)
print('beta:',beta)

In [ ]:
assert np.allclose(beta, np.array([ 94.88442777,  -0.49064841]))

**Exercício:** Agora iremos avaliar a qualidade da regressão. Utilizando os parâmetros obtidos no passo anterior, calcule o desvio

$$
D = \sum_{i=1}^n (\hat  y^{(i)} - y^{(i)})^2
$$

In [ ]:
def calculaDesvio(x, y, beta):
    ### Passo 1: obtenha o número de observações n (~1 linha)
    n = x.shape[0]

    ### Passo 2: calcule o vetor de predições yhat (~1 a 2 linhas)
    yhat = beta[1] * x + beta[0]
    
    ### Passo 3: calcule o desvio (~1 a 3 linhas)
    desvio = np.sum((yhat - y)**2)
    
    return desvio

In [ ]:
assert np.round(calculaDesvio(x,y,beta),3) == 7456.811

**Exercício:** Finalmente, iremos escrever um laço para avaliar qual das variáveis independentes retorna o **menor** desvio quando usada na regressão linear simples.

In [ ]:
# Passo 1: obtenha o número de colunas (variáveis independentes) sobre os quais vamos iterar (~1 linha)
ncols = dados.shape[1]

# inicializando variaveis min_desvio e melhor_coluna
min_desvio = np.inf
melhor_coluna = 0

for j in range(ncols):
    # Passo 2: obtenha x, calcule os parametros beta da regressao e entao calcule o desvio (~3 linhas)
    x = dados[:,j]
    beta = regressaoLinearSimples(x, y)
    desvio = calculaDesvio(x, y, beta)
    
    print('Coluna: {}, Desvio: {}'.format(j, desvio))
    
    # Passo 3: atualize as variáveis min_desvio e melhor_coluna (~3 linhas)
    if desvio < min_desvio:
        min_desvio = desvio
        melhor_coluna = j

# Passo 4: imprima o nome da melhor coluna utilizando a variável col_names (~1 linha)
print('Melhor coluna:', col_names[melhor_coluna])

In [ ]:
### testes ocultos
